In [14]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions.{udf}
import org.apache.spark.sql.DataFrame
import spark.implicits._

import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.DataFrame
import spark.implicits._


In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 3.0.0


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath: String = /home/jovyan/data/events-stream
modelPath: String = /home/jovyan/models/spark-ml-model
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(tweet,StringType,true))
inputDF: org.apache.spark.sql.DataFrame = [tweet: string]


In [4]:
val model = PipelineModel.load(modelPath)
val predictionsDF = model.transform(inputDF)

model: org.apache.spark.ml.PipelineModel = pipeline_bad5478c033e
predictionsDF: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 4 more fields]


In [13]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show(10, false)
}.start()

res5: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@a3bf791


+------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------+----------------------------------------+----------+
|tweet                                                                                                                                     |words                                                                                                                                                              |features                                                                                                          

In [18]:
val Probability_new = udf((probability: org.apache.spark.ml.linalg.Vector) => probability(1))
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF
        .withColumn("probability_new", Probability_new($"probability"))
        .show
}.start()

Probability_new: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4365/0x000000084169c040@7edbbf3,DoubleType,List(Some(class[value[0]: vector])),None,false,true)
res9: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@47d3c982


+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|    probability_new|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|@AZAAVFC4LIFE tha...|[@azaavfc4life, t...|(1000,[158,226,23...|[4.72867607221385...|[0.47286760722138...|       1.0| 0.5271323927786141|
|Rain is so freaki...|[rain, is, so, fr...|(1000,[150,209,40...|[5.12047631273366...|[0.51204763127336...|       0.0|0.48795236872663345|
|On the way to sou...|[on, the, way, to...|(1000,[17,49,152,...|[5.26229398163917...|[0.52622939816391...|       0.0| 0.4737706018360825|
|Ok I just heard B...|[ok, i, just, hea...|(1000,[84,307,333...|[5.05007758002708...|[0.50500775800270...|       0.0|0.49499224199729114|
|     GudMorNin all! |   [gudmorni